In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from scipy.stats import truncnorm
from numba import jit, njit
from shapely import MultiPoint, Point
from shapely.ops import nearest_points

Read dwelling types

In [2]:
dwellingTypes = pd.read_csv('./Data/DwellingTypesByZone2006.csv')

Read Zonal Profile data

In [3]:
profileData = pd.read_csv('./Data/GTAProfileData.csv')

Create ZoneSystem file which contains the zones to be used in the sim

In [4]:
zoneSystem = pd.DataFrame(data=profileData['DAUID'], columns=['DAUID'], dtype=str)
zoneSystem = zoneSystem[zoneSystem['DAUID'].str.len()==8]

Read Shapefile Zones

In [5]:
zoneShape:gpd.GeoDataFrame = gpd.read_file('./Data/TorontoZoneShape/TorontoZones_26917.shp')
zoneShape = zoneShape[zoneShape['DAUID'].astype(str).isin(zoneSystem['DAUID'])]

Create Zone Dataframe that holds all data

zoneData = pd.DataFrame(columns=['DAUID', 'Population', 'Employment rate', 'Area', "Total - Structural type of dwelling"])

for i, zone in zoneSystem.iterrows():
    numDwellings=200
    population = 400
    try:numDwellings = dwellingTypes.loc[dwellingTypes['Geography'].astype(str)==zone['DAUID'], 'Total - Structural type of dwelling'].iloc[0]
    except:pass
    try:population = int(profileData.loc[profileData['DAUID'].astype(str)==zone['DAUID'], "Population"].iloc[0])
    except:pass

    zoneData.loc[len(zoneData)] = [zone['DAUID'], population, profileData.loc[profileData['DAUID'].astype(str)==zone['DAUID'], "Employment rate"].iloc[0], float(zoneShape.loc[zoneShape['DAUID'].astype(str)==zone['DAUID'], 'geometry'].iloc[0].area), numDwellings]

zoneData.to_csv('./Data/combinedZoneData.csv', index=False)

<span style="font-size:5em;">CREATE PERSONS</span>

In [6]:
zoneData = pd.read_csv('./Data/combinedZoneData.csv')

Income generator

In [7]:
def get_truncated_normal(mean=0, sd=1, low=0, upp=10):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)


In [8]:
salary_distribution_employed = get_truncated_normal(mean=70000, sd=40000, low=30000, upp=200000)
salary_distribution_retired = get_truncated_normal(mean=40000, sd=20000, low=15000, upp=70000)

In [9]:
totalPersons = zoneData['Population'].astype(float).sum()

In [10]:
employment_incomes = salary_distribution_employed.rvs(int(totalPersons*.8))
retired_incomes = salary_distribution_retired.rvs(int(totalPersons*.4))

<h3>synthesize persons<h3/>

initial person counts per zone

In [11]:
personZoneCounts = np.empty([0,2], dtype=np.float64)
for i, zone in zoneData.iterrows():
    personZoneCounts = np.r_[personZoneCounts, [[np.float64(zone['DAUID']), np.float64(zone['Population'])]]]

In [12]:
personZoneCounts

array([[3.5190002e+07, 7.9000000e+02],
       [3.5190003e+07, 3.8400000e+02],
       [3.5190004e+07, 6.8700000e+02],
       ...,
       [3.5250973e+07, 4.9900000e+02],
       [3.5250974e+07, 4.7700000e+02],
       [3.5250975e+07, 4.7600000e+02]])

Create numba function to synthesize persons

In [13]:
@njit
def createPersons(personCounts):
    p = np.empty((0,10))
    for zoneNum in range(personCounts.shape[0]):
        print(zoneNum)
        numPeopleInZone = personCounts[zoneNum][1]
        for person in range(numPeopleInZone):
            age = np.random.randint(0,100)
            gender = np.random.randint(0,2)
            # 0:SINGLE 1:MARRIED 2:CHILD
            relationship = 2 if age < 18 else np.random.randint(0, 2)
            occupation = 0 if age < 4 \
                    else 4 if age > 65 \
                    else 3 if age < 22 \
                    else 1 if np.random.random() < .70 \
                    else 2
            occupation_type = 1
            workplace = -1
            income=-1
            if occupation==1:
                income = employment_incomes[np.random.randint(0,int(totalPersons*.9))]
            elif occupation==4:
                income = retired_incomes[np.random.randint(0,int(totalPersons*.1))]
            
            schoolplace=-1
            
            p = np.append(p, np.array([[personZoneCounts[zoneNum][0], -1, age, gender, relationship, occupation, occupation_type, workplace, income, schoolplace]]), axis=0)
        print(zoneNum)
    return p
    

Create dataframe of persons using the c-compiled function above to create the data

In [14]:
persons = pd.DataFrame(data=createPersons(personZoneCounts), columns=['id', 'hhId', 'age', 'gender', 'relationship', 'occupation', 'occupation_type', 'workplace', 'income', 'schoolplace'])

0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
12
12
13
13
14
14
15
15
16
16
17
17
18
18
19
19
20
20
21
21
22
22
23
23
24
24
25
25
26
26
27
27
28
28
29
29
30
30
31
31
32
32
33
33
34
34
35
35
36
36
37
37
38
38
39
39
40
40
41
41
42
42
43
43
44
44
45
45
46
46
47
47
48
48
49
49
50
50
51
51
52
52
53
53
54
54
55
55
56
56
57
57
58
58
59
59
60
60
61
61
62
62
63
63
64
64
65
65
66
66
67
67
68
68
69
69
70
70
71
71
72
72
73
73
74
74
75
75
76
76
77
77
78
78
79
79
80
80
81
81
82
82
83
83
84
84
85
85
86
86
87
87
88
88
89
89
90
90
91
91
92
92
93
93
94
94
95
95
96
96
97
97
98
98
99
99
100
100
101
101
102
102
103
103
104
104
105
105
106
106
107
107
108
108
109
109
110
110
111
111
112
112
113
113
114
114
115
115
116
116
117
117
118
118
119
119
120
120
121
121
122
122
123
123
124
124
125
125
126
126
127
127
128
128
129
129
130
130
131
131
132
132
133
133
134
134
135
135
136
136
137
137
138
138
139
139
140
140
141
141
142
142
143
143
144
144
145
145
146
146
147
147
148
148
149
149
150
150
151
151
152


Clean up varaibles

In [ ]:
# del employment_incomes
# del retired_incomes
# del personZoneCounts
# del profileData
# del dwellingTypes